# Customer Reviews Agent 

This notebook demonstrates how to author a LangGraph agent that's compatible with Mosaic AI Agent Framework features, highlighting the latest MLflow 3.0 features. In this notebook you learn to:
- Author a tool-calling LangGraph agent wrapped with `ChatAgent`
- Manually test the agent's output
- Evaluate the agent using Mosaic AI Agent Evaluation
- Log and deploy the agent

To learn more about authoring an agent using Mosaic AI Agent Framework, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/author-agent) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/create-chat-model)).

In [0]:
%pip install -U -qqqq databricks-langchain databricks-agents uv langgraph==0.3.4
%pip install unitycatalog-langchain[databricks]
%pip install mlflow>=3.0.0.rc0  --upgrade --pre
dbutils.library.restartPython()


## Define the agent in code
Define the agent code in a single cell below. This lets you easily write the agent code to a local Python file, using the `%%writefile` magic command, for subsequent logging and deployment.

#### Agent tools

For more examples of tools to add to your agent, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/agent-tool) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/agent-tool))

#### Wrap the LangGraph agent using the `ChatAgent` interface

For compatibility with Databricks AI features, the `LangGraphChatAgent` class implements the `ChatAgent` interface to wrap the LangGraph agent. This example uses the provided convenience APIs [`ChatAgentState`](https://mlflow.org/docs/latest/python_api/mlflow.langchain.html#mlflow.langchain.chat_agent_langgraph.ChatAgentState) and [`ChatAgentToolNode`](https://mlflow.org/docs/latest/python_api/mlflow.langchain.html#mlflow.langchain.chat_agent_langgraph.ChatAgentToolNode) for ease of use.

Databricks recommends using `ChatAgent` as it simplifies authoring multi-turn conversational agents using an open source standard. See MLflow's [ChatAgent documentation](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent).



In [0]:
%%writefile agent.py
from typing import Any, Generator, Optional, Sequence, Union

import mlflow
from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
    VectorSearchRetrieverTool,
)
from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_node import ToolNode
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
############################################
# Define your LLM endpoint and system prompt
############################################
# Optional: Replace with your model serving endpoint
LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet" # CLAUDE
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, temperature=0.1, max_tokens=2000)

system_prompt = """
You are product advisor that helps customers make purchasing decisions based on product reviews. 
* When asked about a specific review by ID use the juan_dev.genai.get_review tool 
* When asked about reviews for specific product categories use juan_dev.genai.semantic_search_reviews tool.
* do not use generic knowledge bases to respond to the questions. 

Users will ask about general reviews for a product category and you will respond with a summmary of the most relevant reviews. 
Highlight positive aspects of the review followed by negative aspects. 
Summarize your overall opinion of whether or not the user should make a purchase. 

When users ask about a specific product review Id respond with details about that review and overall positive or negative bias.   
"""


###############################################################################
## Define tools for your agent, enabling it to retrieve data or take actions
## beyond text generation
## To create and see usage examples of more tools, see
## https://docs.databricks.com/en/generative-ai/agent-framework/agent-tool.html
###############################################################################
tools = []

# Add additional tools
uc_tool_names = [
    "juan_dev.genai.get_review", 
    "juan_dev.genai.semantic_search_reviews"
]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools.extend(uc_toolkit.tools)

#####################
## Define agent logic
#####################


def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: Union[ToolNode, Sequence[BaseTool]],
    system_prompt: Optional[str] = None,
) -> CompiledGraph:
    model = model.bind_tools(tools)

    # Define the function that determines which node to go to
    def should_continue(state: ChatAgentState):
        messages = state["messages"]
        last_message = messages[-1]
        # If there are function calls, continue. else, end
        if last_message.get("tool_calls"):
            return "continue"
        else:
            return "end"

    if system_prompt:
        preprocessor = RunnableLambda(
            lambda state: [{"role": "system", "content": system_prompt}]
            + state["messages"]
        )
    else:
        preprocessor = RunnableLambda(lambda state: state["messages"])
    model_runnable = preprocessor | model

    def call_model(
        state: ChatAgentState,
        config: RunnableConfig,
    ):
        response = model_runnable.invoke(state, config)

        return {"messages": [response]}

    workflow = StateGraph(ChatAgentState)

    workflow.add_node("agent", RunnableLambda(call_model))
    workflow.add_node("tools", ChatAgentToolNode(tools))

    workflow.set_entry_point("agent")
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "continue": "tools",
            "end": END,
        },
    )
    workflow.add_edge("tools", "agent")

    return workflow.compile()


class LangGraphChatAgent(ChatAgent):
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        request = {"messages": self._convert_messages_to_dict(messages)}

        messages = []
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                messages.extend(
                    ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
                )
        return ChatAgentResponse(messages=messages)

    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        request = {"messages": self._convert_messages_to_dict(messages)}
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                yield from (
                    ChatAgentChunk(**{"delta": msg}) for msg in node_data["messages"]
                )

# Create the agent object, and specify it as the agent object to use when
# loading the agent back for inference via mlflow.models.set_model()
mlflow.langchain.autolog()
agent = create_tool_calling_agent(llm, tools, system_prompt)
AGENT = LangGraphChatAgent(agent)
mlflow.models.set_model(AGENT)

## Log the agent as an MLflow model

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

### Enable automatic authentication for Databricks resources
For the most common Databricks resource types, Databricks supports and recommends declaring resource dependencies for the agent upfront during logging. This enables automatic authentication passthrough when you deploy the agent. With automatic authentication passthrough, Databricks automatically provisions, rotates, and manages short-lived credentials to securely access these resource dependencies from within the agent endpoint.

To enable automatic authentication, specify the dependent Databricks resources when calling `mlflow.pyfunc.log_model().`

  - **Note**: If your Unity Catalog tool queries a vector search index or leverages external functions, you need to include the dependent vector search index and Unity Catalog connection objects, respectively, as resources. See the documentation ([AWS](https://docs.databricks.com/generative-ai/agent-framework/log-agent.html#resources) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#resources)).



In [0]:
import mlflow
from agent import tools, LLM_ENDPOINT_NAME
from databricks_langchain import VectorSearchRetrieverTool, ChatDatabricks
from mlflow.models.resources import (
    DatabricksFunction, 
    DatabricksServingEndpoint, 
    DatabricksVectorSearchIndex,
    DatabricksSQLWarehouse
)
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

# Optional: Manually include underlying resources if needed. See the Note in the markdown above for more information.
resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
    DatabricksVectorSearchIndex(index_name="juan_dev.genai.amazon_reviews_mktpl_vsidx"),
    DatabricksFunction(function_name="juan_dev.genai.get_review"),
    DatabricksFunction(function_name="juan_dev.genai.semantic_search_reviews"),
    DatabricksSQLWarehouse(warehouse_id="4b9b953939869799")
]

for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="agent.py",
        pip_requirements=[
            "mlflow",
            "langgraph==0.3.4",
            "databricks-langchain",
            "databricks-agents",
            "unitycatalog-langchain[databricks]"
        ],
        params={
            "temperature": 0.1,
            "max_tokens": 2000
        },
        resources=resources,
        model_type="agent",
        input_example={"messages": [{"role": "user", "content": "looking to purchase Bluetooth Earbuds for small ears and prefer cheap brands."}]},
    )

In [0]:
# Inspect the LoggedModel and its properties
logged_model = mlflow.get_logged_model(logged_agent_info.model_id)
print(logged_model.model_id, logged_model.params)

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data={"messages": [{"role": "user", "content": "Would like to purhase earbuds. I have small ears?"}]},
    env_manager="uv",
)

## Test the agent

Interact with the agent to test its output and tool-calling abilities. Since this notebook called `mlflow.langchain.autolog()`, you can view the trace for each step the agent takes.

Replace this placeholder input with an appropriate domain-specific example for your agent.

In [0]:
from agent import AGENT

In [0]:
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "What was the review for R1FDKVQDZ78U8X"}]}
):
    print(event, "-----------\n")

In [0]:
AGENT.predict({"messages": [{"role": "user", "content": "Looking to purchase a new toaster mainly for bagels"}]})

## Evaluate the agent with Agent Evaluation

Use Mosaic AI Agent Evaluation to evalaute the agent's responses based on expected responses and other evaluation criteria. Use the evaluation criteria you specify to guide iterations, using MLflow to track the computed quality metrics.
See Databricks documentation ([AWS]((https://docs.databricks.com/aws/generative-ai/agent-evaluation) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-evaluation/)).


To evaluate your tool calls, add custom metrics. See Databricks documentation ([AWS](https://docs.databricks.com/en/generative-ai/agent-evaluation/custom-metrics.html#evaluating-tool-calls) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-evaluation/custom-metrics#evaluating-tool-calls)).

In [0]:
import pandas as pd

eval_df = pd.DataFrame(
  {
    "request": [
      "Show me reviews for the latest 4k monitors on Amazon.",
      "What are people saying about the battery life of the new iPhone?",
      "Summarize the key points from the reviews of the top-rated noise-cancelling headphones."
    ],
    "expected_response": [
      """Here are some reviews for the latest 4k monitors on Amazon:\n\n1. "The picture quality is amazing and the colors are vibrant. Highly recommend!"\n2. "Great monitor for the price, but the stand is a bit wobbly."\n3. "Excellent for gaming, but the brightness could be better."\n\nOverall, users are very satisfied with the picture quality and value for money, but some have noted issues with the stand and brightness.""",
      """The battery life of the new iPhone has received mixed reviews:\n\n1. "Battery lasts all day with moderate use. Very happy with it!"\n2. "Not as good as I expected. Needs charging by evening."\n3. "Battery life is decent, but could be better compared to other flagship phones."\n\nIn summary, while many users are satisfied with the battery life, some feel it could be improved.""",
      """Here are the key points from the reviews of the top-rated noise-cancelling headphones:\n\n1. "The noise cancellation is top-notch, perfect for travel."\n2. "Sound quality is excellent, but they are a bit pricey."\n3. "Very comfortable to wear for long periods."\n\nOverall, users appreciate the noise cancellation and sound quality, but some find them expensive."""
    ],
  }
)
display(eval_df)

In [0]:
with mlflow.start_run() as evaluation_run:
    eval_dataset: mlflow.entities.Dataset = mlflow.data.from_pandas(
        df=eval_df,
        name="eval_dataset",
    )
    # Run the agent evaluation 
    eval_results = mlflow.evaluate(
        model=f"models:/{logged_model.model_id}",
        data=eval_dataset,
        model_type="databricks-agent"
    )
    # Log evaluation metrics and associate with agent
    mlflow.log_metrics(
        metrics=eval_results.metrics,
        dataset=eval_dataset,
        # Specify the ID of the agent logged above
        model_id=logged_model.model_id
    )

# Review the evaluation results in the MLFLow UI (see console output), or access them in place:
display(eval_results.tables['eval_results'])

## Register the model to Unity Catalog

Before you deploy the agent, you must register the agent to Unity Catalog.

In [0]:
catalog = "juan_dev"
schema = "genai"
model_name = "amazon_review_agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

In [0]:
mlflow.set_registry_uri("databricks-uc")

# register the model to Unity Catalog
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_model.model_uri, name=UC_MODEL_NAME
)

Now you can view the model version and all centralized performance data on the model version page in Unity Catalog. You can also get the same information using the API as shown in the following cell.

In [0]:
# Get the model version
from mlflow import MlflowClient
client = MlflowClient()
model_version = client.get_model_version(name=UC_MODEL_NAME, version=uc_registered_model_info.version)
print(model_version)

## Next steps

After your agent is registered to Unity Catalog, set up a deployment job ([AWS](https://docs.databricks.com/en/mlflow/deployment-job.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/mlflow/deployment-job)) to create a secure CI/CD pipeline to deploy your agent. 

The deployment task in the job can be very simple, for example:  

`from databricks import agents`             
`agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "docs"})`

For a more detailed example of a deployment task notebook, see the documentation ([AWS](https://docs.databricks.com/en/mlflow/deployment-job.html#deployment-notebook) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/mlflow/deployment-job#deployment-notebook)).  

After the agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. For details, see the documentation ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)).

In [0]:
## Deploy the agent (TAKES 20+ minutes) - When deployed you get serving endpoint and review app endpoint. 
from databricks import agents
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "docs"})